# Financial Analyst Agent for Annual Report Writing

In this demo, we introduce an agent that can analyze financial report.

In [ ]:
import os
import autogen
from autogen.cache import Cache

from finrobot.utils import register_keys_from_json
from finrobot.toolkits import register_toolkits
from finrobot.functional import (
    ReportChartUtils,
    ReportAnalysisUtils,
    IPythonUtils,
    ReportLabUtils,
    TextUtils,
)
from finrobot.data_source import FMPUtils
from textwrap import dedent


os.environ["FMP_API_KEY"] = "YOUR_FMP_API_KEY"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

work_dir = r"D:\STAT-GR5398-Spring-2026\FinRobot-Equity-Research-Group\Assignment1\source_code\report"
os.makedirs(work_dir, exist_ok=True)

print("FMP key ok:", os.environ["FMP_API_KEY"][:8], "...")
print("OpenAI key ok:", os.environ["OPENAI_API_KEY"][:8], "...")
print("work_dir:", work_dir)

After importing all the necessary packages and functions, we also need the config for OpenAI & SecApi & FMPApi here.
- for openai configuration, rename OAI_CONFIG_LIST_sample to OAI_CONFIG_LIST and replace the api keys
- for Sec_api & FMP_api configuration, rename config_api_keys_sample to config_api_keys and replace the api keys

In [ ]:
import os

config_list = [{
    "model": "gpt-4.1",
    "api_key": os.environ["OPENAI_API_KEY"],
    "base_url": "https://api.openai.com/v1",
}]
llm_config = {
    "config_list": config_list,
    "timeout": 120,
    # "temperature": 0 # for debug convenience
    "temperature": 0.5,
}
#register_keys_from_json("/content/FinRobot/config_api_keys")

# Intermediate results will be saved in this directory
#work_dir = "/content/FinRobot/report"
#os.makedirs(work_dir, exist_ok=True)

For this task, we need:
- A user proxy to execute python functions and control the conversations.
- An expert agent who is proficient in financial analytical writing.
- A shadow/inner-assistant to handle isolated long-context Q&As. (Because we dont want to keep the sec files in the chat history.)
In the following cell, we define the agents, and equip them with necessary tools.

In [ ]:
system_message = dedent(
    f"""
    Role: Expert Investor
    Department: Finance
    Primary Responsibility: Generation of Customized Financial Analysis Reports

    Role Description:
    As an Expert Investor within the finance domain, your expertise is harnessed to develop bespoke Financial Analysis Reports that cater to specific client requirements. This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.

    Key Objectives:

    Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.
    Effective Communication: Simplify and effectively convey complex financial narratives, making them accessible and actionable to non-specialist audiences.
    Client Focus: Dynamically tailor reports in response to client feedback, ensuring the final analysis aligns with their strategic objectives.
    Adherence to Excellence: Maintain the highest standards of quality and integrity in report generation, following established benchmarks for analytical rigor.
    Performance Indicators:
    The efficacy of the Financial Analysis Report is measured by its utility in providing clear, actionable insights. This encompasses aiding corporate decision-making, pinpointing areas for operational enhancement, and offering a lucid evaluation of the company's financial health. Success is ultimately reflected in the report's contribution to informed investment decisions and strategic planning.

    Reply TERMINATE when everything is settled.
    """
)
expert = autogen.AssistantAgent(
    name="Expert_Investor",
    system_message=system_message,
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "")
    and x.get("content", "").endswith("TERMINATE"),
)
expert_shadow = autogen.AssistantAgent(
    name="Expert_Investor_Shadow",
    system_message=system_message,
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content","")
    and x.get("content","").endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": work_dir,
        "use_docker": False,
    },
)


In [ ]:
register_toolkits(
    [
        FMPUtils.get_sec_report,  # Retrieve SEC report url and filing date
        IPythonUtils.display_image,  # Display image in IPython
        TextUtils.check_text_length,  # Check text length
        ReportLabUtils.build_annual_report,  # Build annual report in designed pdf format
        ReportAnalysisUtils,  # Expert Knowledge for Report Analysis
        ReportChartUtils,  # Expert Knowledge for Report Chart Plotting
    ],
    expert,
    user_proxy,
)

In this section, we introduce the **nested chat between the expert and the shadow**, which is triggered only when the expert gets response from function execution showing that an writing instruction and related context & resources are ready. The writing would then happen in this separate chat, the shadow (or call it the inner mind of expert) would do the financial writing based on the instruction and turn back to the expert. However, this chat is muted due to long context.

In [ ]:
def order_trigger(sender):
    # Check if the last message contains the path to the instruction text file
    return "instruction & resources saved to" in sender.last_message()["content"]


def order_message(recipient, messages, sender, config):
    # Extract the path to the instruction text file from the last message
    full_order = recipient.chat_messages_for_summary(sender)[-1]["content"]
    txt_path = full_order.replace("instruction & resources saved to ", "").strip()
    txt_path = txt_path.splitlines()[0].strip()
    with open(txt_path, "r") as f:
        instruction = f.read() + "\n\nReply TERMINATE at the end of your response."
    return instruction


# Since 10-K section is not short, we need an extra nested chat to analyze the contents
expert.register_nested_chats(
    [
        {
            "sender": expert,
            "recipient": expert_shadow,
            "message": order_message,
            "summary_method": "last_msg",
            "max_turns": 2,
            "silent": True,  # mute the chat summary
        }
    ],
    trigger=order_trigger,
)

### Resources to understand the financial report
1. income statement: https://online.hbs.edu/blog/post/income-statement-analysis
2. balance sheet: https://online.hbs.edu/blog/post/how-to-read-a-balance-sheet
3. cash flow statement: https://online.hbs.edu/blog/post/how-to-read-a-cash-flow-statement
4. Annual report: https://online.hbs.edu/blog/post/how-to-read-an-annual-report

An annual report typically consists of:
1. Letters to shareholders: These documents provide a broad overview of the company’s activities and performance over the course of the year, as well as a reflection on its general business environment. An annual report usually includes a shareholder letter from the CEO or president, and may also contain letters from other key figures, such as the CFO.
2. [section 7] Management’s discussion and analysis (MD&A): This is a detailed analysis of the company’s performance, as conducted by its executives.
3. [section 8] Audited financial statements: These are financial documents that detail the company’s financial performance. Commonly included statements include balance sheets, cash flow statements, income statements, and equity statements.
4. [section 8] A summary of financial data: This refers to any notes or discussions that are pertinent to the financial statements listed above.
5. [section 8] Auditor’s report: This report describes whether the company has complied with generally accepted accounting principles (GAAP) in preparing its financial statements.
6. Accounting policies: This is an overview of the policies the company’s leadership team relied upon in preparing the annual report and financial statements.


Answer the following questions:
1. Whether it’s able to pay debts as they come due
2. Its profits and/or losses year over year
3. If and how it’s grown over time
4. What it requires to maintain or expand its business
5. Operational expenses compared to generated revenues

---

Now, let's see how our agent does.



In [ ]:
company = "AMD"#this can be changed
competitors = ["AAPL", "NVDA", "INTC", "GOOGL"]   
fyear = "2024"
work_dir = r"D:\STAT-GR5398-Spring-2026\FinRobot-Equity-Research-Group\Assignment1\source_code\report\amd"
os.makedirs(work_dir, exist_ok=True)


task = dedent(
    f"""
    With the tools you've been provided, write an annual report based on {company}'s and{competitors}'s{fyear} 10-k report, format it into a pdf.
    Pay attention to the followings:
    - Explicitly explain your working plan before you kick off.
    - Use tools one by one for clarity, especially when asking for instructions.
    - All your file operations should be done in "{work_dir}".
    - Display any image in the chat once generated.
    - For competitors analysis, strictly follow my prompt and use data only from the financial metics table, do not use similar sentences in other sections, delete similar setence, classify it into either of the two. The last sentence always talks about the Discuss how {company}’s performance over these years and across these metrics might justify or contradict its current market valuation (as reflected in the EV/EBITDA ratio).
    - Each paragraph in the first page(business overview, market position and operating results) should be between 150 and 160 words, each paragraph in the second page(risk assessment and competitors analysis) should be between 500 and 600 words, don't generate the pdf until this is explicitly fulfilled.
"""
)

with Cache.disk() as cache:
    user_proxy.initiate_chat(
        recipient=expert,
        message=task,
        clear_history=True,
        max_turns=500,
        summary_method="last_msg",
    )

In [ ]:
import os
from pathlib import Path
from datetime import datetime

work_dir = Path(r"D:\STAT-GR5398-Spring-2026\FinRobot-Equity-Research-Group\Assignment1\source_code\report\googl")

for f in sorted(work_dir.iterdir()):
    if f.is_file():
        mtime = datetime.fromtimestamp(f.stat().st_mtime)
        print(f"{f.name:40s}  {mtime}")


The Rest Cells are optional, simply used to show the generated PDF

In [ ]:
import os
from pathlib import Path

work_dir = Path(r"D:\STAT-GR5398-Spring-2026\FinRobot-Equity-Research-Group\Assignment1\source_code\report\aapl")
out_pdf = work_dir / "aapl_2024_annual_report.pdf"

def read_text_if_exists(path: Path) -> str:
    if not path or not path.exists():
        return ""
    try:
        return path.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        return path.read_text(encoding="utf-8", errors="ignore")

def find_first(patterns):
    files = [p for p in work_dir.iterdir() if p.is_file()]
    # exact name first
    for name in patterns:
        p = work_dir / name
        if p.exists():
            return p
    # contains match
    for pat in patterns:
        for f in files:
            if pat.lower() in f.name.lower():
                return f
    return None

# tolerant to various naming used by finrobot notebook
txt_operating = find_first(["aapl_operating_results.txt","aapl_operating", "operating_results"])
txt_comp      = find_first(["aapl_competitors_analysis.txt","aapl_competitors", "competitors_analysis"])
txt_income    = find_first(["aapl_income_statement.txt","aapl_income_stmt.txt","income_statement","income_stmt"])
txt_segment   = find_first(["aapl_segment_analysis.txt","aapl_segment_stmt.txt","segment_analysis","segment_stmt"])

png_share = find_first(["aapl_share_performance.png","share_performance"])
png_peeps = find_first(["aapl_pe_eps_performance.png","pe_eps_performance","pe_eps"])

sections = [
    ("Operating Results", read_text_if_exists(txt_operating)),
    ("Income Statement / Key Financials", read_text_if_exists(txt_income)),
    ("Segment / Business Breakdown", read_text_if_exists(txt_segment)),
    ("Competitors Analysis", read_text_if_exists(txt_comp)),
]

from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

W, H = letter
margin = 0.75 * inch
line_h = 12

def draw_wrapped_text(c, text, x, y, max_width, font="Times-Roman", size=10):
    c.setFont(font, size)
    words = (text or "").replace("\r\n", "\n").split()
    if not words:
        return y
    line = ""
    for w in words:
        test = (line + " " + w).strip()
        if c.stringWidth(test, font, size) <= max_width:
            line = test
        else:
            c.drawString(x, y, line)
            y -= line_h
            line = w
            if y < margin:
                c.showPage()
                c.setFont(font, size)
                y = H - margin
    if line:
        c.drawString(x, y, line)
        y -= line_h
    return y

c = canvas.Canvas(str(out_pdf), pagesize=letter)

# Title
c.setFont("Times-Bold", 18)
c.drawString(margin, H - margin, "Apple Inc. (AAPL) FY2024 Annual Report")
c.setFont("Times-Roman", 11)
c.drawString(margin, H - margin - 22, f"Source folder: {work_dir}")
c.drawString(margin, H - margin - 38, "Compiled from FinRobot intermediate outputs (txt/png).")
c.showPage()

# Charts
y = H - margin
c.setFont("Times-Bold", 14)
c.drawString(margin, y, "Charts")
y -= 18

def draw_image(img_path, title):
    nonlocal_y = None
    return

def draw_image_if_exists(img_path, title):
    global y
    if not img_path or not Path(img_path).exists():
        return
    c.setFont("Times-Bold", 11)
    c.drawString(margin, y, title)
    y -= 14
    try:
        img = ImageReader(str(img_path))
        max_w = W - 2 * margin
        draw_h = max_w * 0.55
        if y - draw_h < margin:
            c.showPage()
            y = H - margin
        c.drawImage(img, margin, y - draw_h, width=max_w, height=draw_h, preserveAspectRatio=True, anchor='nw')
        y -= (draw_h + 18)
    except Exception as e:
        c.setFont("Times-Roman", 10)
        c.drawString(margin, y, f"[Could not render image {Path(img_path).name}: {e}]")
        y -= 14

draw_image_if_exists(png_share, "Share Performance")
draw_image_if_exists(png_peeps, "PE / EPS Performance")
c.showPage()

# Text sections
for title, body in sections:
    c.setFont("Times-Bold", 14)
    c.drawString(margin, H - margin, title)
    y = H - margin - 20
    if not (body or "").strip():
        c.setFont("Times-Roman", 11)
        c.drawString(margin, y, "[No content found for this section in the folder.]")
        c.showPage()
        continue
    y = draw_wrapped_text(c, body, margin, y, W - 2 * margin, font="Times-Roman", size=10)
    c.showPage()

c.save()

print("✅ PDF created:", out_pdf)
print("files now:", sorted([p.name for p in work_dir.iterdir()]))



In [ ]:
pip install reportlab

In [ ]:
!pip install PyMuPDF

In [ ]:
import io
import fitz
from PIL import Image


pdf = fitz.open("/content/FinRobot/report/NextEra_Annual_Report_2024.pdf")
page = pdf.load_page(0)
pix = page.get_pixmap()

# Convert the Pixmap to a PIL Image
img = Image.open(io.BytesIO(pix.tobytes("png")))
display(img)